### Using this to get the entries with no consensus

In [2]:
import pandas as pd
from operator import itemgetter
import csv
import numpy as np
df = pd.read_csv (r"../data/labelbox_v2-1_export.csv")

### remove unnecessary data

In [3]:
df.sort_values(by=["External ID"], inplace=True)
df = df.reset_index() 
for index, row in df.iterrows():
    if row["Label"] == "{\"select_which_leadership_style_appear_in_the_text,_if_any:\":\"transactional\"}":
        df.iloc[index, 4] = "TC"
    if row["Label"] == "{\"select_which_leadership_style_appear_in_the_text,_if_any:\":\"transformational\"}":
        df.iloc[index, 4] = "TF"
    if row["Label"] == "{\"select_which_leadership_style_appear_in_the_text,_if_any:\":\"uncertain\"}":
        df.iloc[index, 4] = "IDK"
    if row["Label"] == "{\"select_which_leadership_style_appear_in_the_text,_if_any:\":\"the_leadership_style_is_uncertain\"}":
        df.iloc[index, 4] = "IDK"
    if row["Label"] == "{\"select_which_leadership_style_appear_in_the_text,_if_any:\":\"does_not_show_leadership\"}":
        df.iloc[index, 4] = "NA"
    if row["Label"] == "{\"select_which_leadership_style_appear_in_the_text,_if_any:\":\"none\"}":
        df.iloc[index, 4] = "NA"
    if row["Created By"] == "dnsmith@g.clemson.edu":
        df.iloc[index, 5] = "d"
    if row["Created By"] == "ajames8@g.clemson.edu":
        df.iloc[index, 5] = "b"
    if row["Created By"] == "mschul3@g.clemson.edu":
        df.iloc[index, 5] = "c"
    if row["Created By"] == "tnunama@clemson.edu":
        df.iloc[index, 5] = "a"
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Seconds to Label,External ID,Agreement,Dataset Name,Reviews,View Label
0,796,cjukdqkwg957m0987oze69gnp,cjuepa6czpa230bj4w0cgou1c,Plate making is an important step in the pre-p...,NA,d,Career Readiness - Leadership v2.1,1555454987000,4.746,1001_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...
1,352,cjuioh6mwv5i80987jkd97mhc,cjuepa6csp9r20bj4ruoqids4,Each individual on our team has different meth...,NA,b,Career Readiness - Leadership v2.1,1555352092000,6.667,1001_1,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...
2,558,cjuj0csps2m9q0818cjfq9l31,cjuepa6cup9vk0bj4b5or9qgo,Pointed out areas of improvement to mentor and...,IDK,c,Career Readiness - Leadership v2.1,1555372042000,4.315,1003_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...
3,313,cjuio7695uopi081717ci3wne,cjuepa6crp9mr0bj4wd0v42uu,"""Prior to the Spring Jubilee Festival, we real...",NA,b,Career Readiness - Leadership v2.1,1555351625000,4.152,1005_0,0.0,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...
4,122,cjuie7wj1q17l08711rq01he5,cjuepa6crp9mr0bj4wd0v42uu,"""Prior to the Spring Jubilee Festival, we real...",IDK,c,Career Readiness - Leadership v2.1,1555334862000,9.947,1005_0,0.5,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...


In [4]:
df.groupby("Label").agg("count")

,index,ID,DataRow ID,Labeled Data,Created By,Project Name,Created At,Seconds to Label,External ID,Agreement,Dataset Name,Reviews,View Label
Label,,,,,,,,,,,,,
IDK,224,224,224,224,224,224,224,224,224,68,224,224,224
NA,290,290,290,290,290,290,290,290,290,91,290,290,290
Skip,18,18,18,18,18,18,18,18,18,6,18,18,18
TC,140,140,140,140,140,140,140,140,140,54,140,140,140
TF,208,208,208,208,208,208,208,208,208,83,208,208,208
{},1,1,1,1,1,1,1,1,1,0,1,1,1


### Assign final labels

In [5]:
# indices of labels: 0 = TC, 1 = TF, 2 = IDK, 3 = NA
styles = {
    "IDK" : 0,
    "NA" : 1,
    "TC" : 2,
    "TF" : 3
}

# labels[0] is the label, label[1] is the count, label[2] is the most authoritative selection
labels = [["IDK", 0, ""], ["NA", 0, ""], ["TC", 0, ""], ["TF", 0, ""]]
next_id = ""

#add column for final label
df["Final Label"] = ""

for i in range(len(df.index)):
    label = df.iloc[i, 4]
    person = df.iloc[i, 5]
    curr_id = df.iloc[i, 9]
    next_id = "" if i >= len(df.index)-1 else df.iloc[i+1, 9]
    if label == "Skip" or label == "{}":
        continue
    else:
        labels[styles[label]][1] += 1
        if person < labels[styles[label]][2] or labels[styles[label]][2] == "":
            labels[styles[label]][2] = person
    if curr_id != next_id:
        labels.sort(key=itemgetter(1), reverse=True)
        # handle conflicts
        # case 0
        if labels[0][1] == labels[1][1]:
            #case 1
            if labels[2][1] == 1:
                if labels[3][1] == 1:
                    labels.sort(key=itemgetter(2))
                    df.iloc[i, 14] = labels[0][0]
                else:
                    segment = sorted(labels[:3], key=itemgetter(2))
                    df.iloc[i, 14] = segment[0][0]
            else:
                segment = sorted(labels[:2], key=itemgetter(2))
                df.iloc[i, 14] = segment[0][0]
        else:
            df.iloc[i, 14] = labels[0][0]
        for j in labels:
            j[1] = 0
            j[2] = ""
        #the below works because this whole cell of code depends on the sorting IDK < NA < TC < TF
        labels.sort(key=itemgetter(0))

In [6]:
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Seconds to Label,External ID,Agreement,Dataset Name,Reviews,View Label,Final Label
0,796,cjukdqkwg957m0987oze69gnp,cjuepa6czpa230bj4w0cgou1c,Plate making is an important step in the pre-p...,NA,d,Career Readiness - Leadership v2.1,1555454987000,4.746,1001_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
1,352,cjuioh6mwv5i80987jkd97mhc,cjuepa6csp9r20bj4ruoqids4,Each individual on our team has different meth...,NA,b,Career Readiness - Leadership v2.1,1555352092000,6.667,1001_1,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
2,558,cjuj0csps2m9q0818cjfq9l31,cjuepa6cup9vk0bj4b5or9qgo,Pointed out areas of improvement to mentor and...,IDK,c,Career Readiness - Leadership v2.1,1555372042000,4.315,1003_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,IDK
3,313,cjuio7695uopi081717ci3wne,cjuepa6crp9mr0bj4wd0v42uu,"""Prior to the Spring Jubilee Festival, we real...",NA,b,Career Readiness - Leadership v2.1,1555351625000,4.152,1005_0,0.0,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,
4,122,cjuie7wj1q17l08711rq01he5,cjuepa6crp9mr0bj4wd0v42uu,"""Prior to the Spring Jubilee Festival, we real...",IDK,c,Career Readiness - Leadership v2.1,1555334862000,9.947,1005_0,0.5,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,


In [7]:
df.loc[:,"Labeled Data"] = df.loc[:,"Labeled Data"].str.replace('"', "")
df["Final Label"].replace("", np.nan, inplace=True)
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Seconds to Label,External ID,Agreement,Dataset Name,Reviews,View Label,Final Label
0,796,cjukdqkwg957m0987oze69gnp,cjuepa6czpa230bj4w0cgou1c,Plate making is an important step in the pre-p...,NA,d,Career Readiness - Leadership v2.1,1555454987000,4.746,1001_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
1,352,cjuioh6mwv5i80987jkd97mhc,cjuepa6csp9r20bj4ruoqids4,Each individual on our team has different meth...,NA,b,Career Readiness - Leadership v2.1,1555352092000,6.667,1001_1,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
2,558,cjuj0csps2m9q0818cjfq9l31,cjuepa6cup9vk0bj4b5or9qgo,Pointed out areas of improvement to mentor and...,IDK,c,Career Readiness - Leadership v2.1,1555372042000,4.315,1003_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,IDK
3,313,cjuio7695uopi081717ci3wne,cjuepa6crp9mr0bj4wd0v42uu,"Prior to the Spring Jubilee Festival, we reali...",NA,b,Career Readiness - Leadership v2.1,1555351625000,4.152,1005_0,0.0,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NaN
4,122,cjuie7wj1q17l08711rq01he5,cjuepa6crp9mr0bj4wd0v42uu,"Prior to the Spring Jubilee Festival, we reali...",IDK,c,Career Readiness - Leadership v2.1,1555334862000,9.947,1005_0,0.5,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NaN


In [8]:
df = df.dropna(subset=['Final Label'], axis=0).reset_index(drop=True)
df.head()

,index,ID,DataRow ID,Labeled Data,Label,Created By,Project Name,Created At,Seconds to Label,External ID,Agreement,Dataset Name,Reviews,View Label,Final Label
0,796,cjukdqkwg957m0987oze69gnp,cjuepa6czpa230bj4w0cgou1c,Plate making is an important step in the pre-p...,NA,d,Career Readiness - Leadership v2.1,1555454987000,4.746,1001_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
1,352,cjuioh6mwv5i80987jkd97mhc,cjuepa6csp9r20bj4ruoqids4,Each individual on our team has different meth...,NA,b,Career Readiness - Leadership v2.1,1555352092000,6.667,1001_1,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,NA
2,558,cjuj0csps2m9q0818cjfq9l31,cjuepa6cup9vk0bj4b5or9qgo,Pointed out areas of improvement to mentor and...,IDK,c,Career Readiness - Leadership v2.1,1555372042000,4.315,1003_0,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,IDK
3,123,cjuie7wj3lchr0944y5eg9x5c,cjuepa6crp9mr0bj4wd0v42uu,"Prior to the Spring Jubilee Festival, we reali...",IDK,c,Career Readiness - Leadership v2.1,1555334862000,25.613,1005_0,0.5,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,IDK
4,333,cjuiobuwsqi950944s6ib3bzv,cjuepa6csp9qb0bj4p9oy0di8,Many returning artists were frustrated with th...,TF,c,Career Readiness - Leadership v2.1,1555351843000,3.952,1005_1,NaN,student_responses_sentences.csv,[],https://image-segmentation-v4.labelbox.com?pro...,TF


In [9]:
df_clean = df.loc[:,['Labeled Data', 'Final Label']]
df_clean.to_csv('labelbox_v2-1_clean.csv', index=False, quoting = csv.QUOTE_NONNUMERIC)

### Trying to make a csv file for the process_for_watson script

In [11]:
df_clean.to_csv('labelbox_v2-1_watson.csv', index=False, sep='|')